<a href="https://colab.research.google.com/github/AIManifest/ComfyUI-Notebook/blob/master/ComfyUI_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<left><font color="orange" size="16"><b>ComfyUI-Notebook</b></font></left>

In [2]:
#@title $\color{orange}{\textsf{Setup HOME Path for Root Working Directory (Optional)}}$
try:
    from google.colab import drive
    gdrive = '/content/drive/'
    drive.mount(gdrive)
    HOME = '/content/'
    print(f'Root Working Directory: {HOME}')
except:
    print(f'Colab not Detected, Changing Paths to Appropriate Root Directory ("/workspace") for Jupyter')
    HOME = '/workspace/'
    print(f'Root Working Directory: {HOME}')

Mounted at /content/drive/
Root Working Directory: /content/


In [5]:
#@title $\color{orange}{\textsf{Run This Cell To Setup Jupyter Notebook/Lab to Run ComfyUI-Notebook}}$
import os
import subprocess

try:
    from google.colab import drive
    HOME = "/content/"
except:
    HOME = "/workspace/"
override_working_dir = False #@param{type:"boolean"}
override_dir = "/content/ComfyUI-Notebook/" #@param{type:"string"}
run_full_setup_for_jupyter = False #@param{type:"boolean"}
install_comfy = True #@param{type:"boolean"}
install_esrgan = False #@param{type:"boolean"}
xformers_version = 'xformers==0.0.20' #@param{type:"string"}

def setup_comfy():
    if run_full_setup_for_jupyter:
        subprocess.run('curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -', shell=True)
        subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'])

        subprocess.run(['apt-get', 'update'])
        subprocess.run(['apt-get', 'install', 'ffmpeg', 'libsm6', 'libxext6', '-y'])
        subprocess.run(['pip', 'install', 'pandas'])

        subprocess.run(['sudo', 'apt-get', 'update'])
        subprocess.run(['sudo', 'apt-get', 'install', 'npm', '-y'])
    if install_comfy:
        os.system("git clone https://github.com/AIManifest/ComfyUI-Notebook.git")
    if install_esrgan:
        os.system("git clone https://github.com/AIManifest/ESRGAN.git")

    WORKING_DIR = os.path.join(HOME, "ComfyUI-Notebook/")
    if override_working_dir:
        WORKING_DIR = override_dir
    os.chdir(WORKING_DIR)
    print(f'Running in Directory: {WORKING_DIR}')
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
    subprocess.run(['pip', 'install', 'triton', xformers_version, 'pytorch-lightning'])

setup_comfy()


Running in Directory: /content/ComfyUI-Notebook/


In [ ]:
#@title $\color{orange}{\textsf{Install Models}}$
import os
import time
import shutil
import requests
from tqdm.auto import tqdm

model_dir = "/content/ComfyUI-Notebook/models/checkpoints/" #@param{type:"string"}
vae_dir = "/content/ComfyUI-Notebook/models/vae/" #@param{type:"string"}
lora_dir = "/content/ComfyUI-Notebook/models/loras/" #@param{type:"string"}

# Add boolean parameters for each model
download_sd_xl_offset_example_lora_1_0 = True #@param {type:"boolean"}
download_sd_xl_base_1_0 = True #@param {type:"boolean"}
download_sd_xl_refiner_1_0 = True #@param {type:"boolean"}
download_sd_xl_base_1_0_0_9vae = True #@param {type:"boolean"}
download_sd_xl_refiner_1_0_0_9vae = True #@param {type:"boolean"}
download_dreamshaperXL10_alpha2XI10 = True #@param {type:"boolean"}

def download_sdxl():
    models_to_download = [
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", lora_dir, "sd_xl_offset_example-lora_1.0.safetensors", download_sd_xl_offset_example_lora_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", model_dir, "sd_xl_base_1.0.safetensors", download_sd_xl_base_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", model_dir, "sd_xl_refiner_1.0.safetensors", download_sd_xl_refiner_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors", model_dir, "sd_xl_base_1.0_0.9vae.safetensors", download_sd_xl_base_1_0_0_9vae),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", model_dir, "sd_xl_refiner_1.0_0.9vae.safetensors", download_sd_xl_refiner_1_0_0_9vae),
        ("https://civitai.com/api/download/models/126688", model_dir, "dreamshaperXL10_alpha2XI10.safetensors", download_dreamshaperXL10_alpha2XI10)
    ]

    username = "AIManifest"
    password = "winterfellCCJP04"

    for model_URL, folder_path, filename, download in models_to_download:
        if not download:
            continue

        if not os.path.exists(folder_path):
            os.makedirs(folder_path, exist_ok=True)

        if not os.path.exists(os.path.join(folder_path, filename)):
            start_time = time.time()

            try:
                auth = (username, password) if username and password else None
                with requests.get(model_URL, stream=True, auth=auth) as r:

                    total_length = int(r.headers.get("Content-Length"))

                    with tqdm.wrapattr(r.raw, "read", total=total_length, desc="..Downloading..", colour='orange') as raw:

                        with open(os.path.join(folder_path, filename), 'wb')as output:
                            shutil.copyfileobj(raw, output)

                end_time = time.time()
                final_time = end_time-start_time
                print(f"\033[38;2;255;192;203mDone! Model {filename} downloaded in {final_time:.2f} seconds!")
                print(f"\033[38;2;255;192;203mFile {filename} downloaded and saved to {folder_path}")
            except Exception as e:
                print(f"An error occurred while downloading the model: {e}")

download_sdxl()

#<left><font color="orange" size="6"><b>Run ComfyUI-Notebook For Web Browser</b></font></left>

In [ ]:
#@title $\color{orange}{\textsf{Run ComfyUI-Notebook For Web Browser}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI-Notebook finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --preview-method "auto"



#<left><font color="orange" size="6"><b>Run ComfyUI-Notebook Notebook System</b></font></left>

##$\color{orange} {\textsf {RUN SDXL -> Single Prompt!}}$


In [ ]:
#@title $\color{orange}{\textsf{Set CudaMallocAsync}}$
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [9]:
#@title $\color{orange} {\textsf {RUN SDXL -> Single Prompt!}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "" #@param{type:"string"}
    negativeprompt = "" #@param{type:"string"}
    #@markdown $\color{orange}{\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='dreamshaperXL10_alpha2XI10.safetensors' #@param{type:"string"}["sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Base Model}}$
    steps = 60 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 50 #@param{type:"number"}

    #refiner model
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown $\color{orange}{\textsf{Image-Batch Settings}}$
    imagewidth = 768 #@param{type:"number"}
    imageheight = 1200   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 7 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown $\color{orange}{\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "karras" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown $\color{orange}{\textsf{Sampler Options}}$
    s_churn = 0. # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.1 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown $\color{orange}{\textsf{CLIP Options}}$
    stop_at_last_layer = -12 #@param{type:"number"}
    width = 4096 #@param{type:"number"}
    height = 4096 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 4096 #@param{type:"number"}
    target_height = 4096 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown $\color{orange}{\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Save Options}}$
    output_folder = "cosmic" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)

with torch.inference_mode():
    try:
        out
        refined_out
    except:
        out = loadsdxl(sdxl_args)
        refined_out = loadsdxlrefiner(sdxl_args)
    model, samples = runsdxl(sdxl_args, out)
    model_management.unload_model(model)
    model_management.unload_model(out[1].patcher)
    sdxlrun.get_device_memory()
    refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
    model_management.unload_model(refinermodel)
    model_management.unload_model(refined_out[1].patcher)
    sdxlrun.get_device_memory()

##$\color{orange} {\textsf {RUN SDXL -> Single Prompt for All Schedulers and Samplers}}$


In [ ]:
#@title $\color{orange}{\textsf{Set CudaMallocAsync}}$
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
#@title $\color{orange} {\textsf {RUN SDXL -> Single Prompt for All Schedulers and Samplers}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = ""
    negativeprompt = ""
    #@markdown $\color{orange}{\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='dreamshaperXL10_alpha2XI10.safetensors' #@param{type:"string"}["sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Base Model}}$
    steps = 60 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 50 #@param{type:"number"}

    #refiner model
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown $\color{orange}{\textsf{Image-Batch Settings}}$
    imagewidth = 768 #@param{type:"number"}
    imageheight = 1200   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 7 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown $\color{orange}{\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "karras" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown $\color{orange}{\textsf{Sampler Options}}$
    s_churn = 0. # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.1 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown $\color{orange}{\textsf{CLIP Options}}$
    stop_at_last_layer = -12 #@param{type:"number"}
    width = 4096 #@param{type:"number"}
    height = 4096 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 4096 #@param{type:"number"}
    target_height = 4096 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown $\color{orange}{\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Save Options}}$
    output_folder = "cosmic" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}

    return locals()

# ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive",]
sampler_names = ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
schedulers = ["normal", "karras", "exponential", "simple", "ddim_uniform"]
randomize_seed = False #@param{type:"boolean"}

for sampler_name in iprogress(sampler_names, colour="synthlite"):
    for scheduler in iprogress(schedulers, colour="cotton_candy"):
        if randomize_seed:
            seed = seed_everything(random.randint(2 ** 64))
        sdxl_args = SDXL_ARGS()
        sdxl_args = SimpleNamespace(**sdxl_args)
        sdxl_args.sampler_name = sampler_name
        sdxl_args.scheduler = scheduler
        sdxl_args.saveprefix = f'{sampler_name}_{scheduler}'
        with torch.inference_mode():
            try:
                out
                refined_out
            except:
                out = loadsdxl(sdxl_args)
                refined_out = loadsdxlrefiner(sdxl_args)

            model, samples = runsdxl(sdxl_args, out)
            model_management.unload_model(model)
            model_management.unload_model(out[1].patcher)
            sdxlrun.get_device_memory()
            refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
            model_management.unload_model(refinermodel)
            model_management.unload_model(refined_out[1].patcher)
            sdxlrun.get_device_memory()


##$\color{orange} {\textsf{Run SDXL -> Multi-Prompt}}$


In [ ]:
#@title $\color{orange}{\textsf{Set CudaMallocAsync}}$
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

'backend:cudaMallocAsync'

In [ ]:
prompts = {
    1: "(dark magic), (grim), ten horns, ten diadems, (intricate details), (hyperdetailed), 8k hdr, high detailed, lot of details, high quality, soft cinematic light, dramatic atmosphere, atmospheric perspective",
    # 2: "an extremely detailed portrait of the last supper with jesus and the disciples by leonardo da vinci in rick and morti art style, picture perfect"
}

negative_prompts = {
    1: "blur, dof, bokeh",
    # 2: "noisy, not sharp, unclear, deformed, disproportionate, hybrid,female, woman, close up, zoomed, zoomed image, low res, low quality, low detail, low dynamic, poorly composed, poorly detailed, poorly proportioned, overblown image",

}

In [ ]:
#@title $\color{orange} {\textsf{Run SDXL -> Multi-Prompt}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = ""
    negativeprompt = ""
    #@markdown $\color{orange}{\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='dreamshaperXL10_alpha2XI10.safetensors' #@param{type:"string"}["sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Base Model}}$
    steps = 60 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 50 #@param{type:"number"}

    #refiner model
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown $\color{orange}{\textsf{Image-Batch Settings}}$
    imagewidth = 768 #@param{type:"number"}
    imageheight = 1200   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 7 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown $\color{orange}{\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "karras" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown $\color{orange}{\textsf{Sampler Options}}$
    s_churn = 0. # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.1 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown $\color{orange}{\textsf{CLIP Options}}$
    stop_at_last_layer = -12 #@param{type:"number"}
    width = 4096 #@param{type:"number"}
    height = 4096 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 4096 #@param{type:"number"}
    target_height = 4096 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown $\color{orange}{\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown $\color{orange}{\textsf{Save Options}}$
    output_folder = "cosmic" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)
randomize_seed = False

key=1
for prompt_id, prompt in prompts.items():
    sdxl_args.prompt = prompt
    sdxl_args.negativeprompt = negative_prompts[key]
    print(f'Running with Prompt: {sdxl_args.prompt}')
    print(f'Running with Negative Prompt: {sdxl_args.negativeprompt}')
    if randomize_seed:
        seed = seed_everything(random.randint(2 ** 64))

    with torch.inference_mode():
        try:
            out
            refined_out
        except:
            out = loadsdxl(sdxl_args)
            refined_out = loadsdxlrefiner(sdxl_args)

        model, samples = runsdxl(sdxl_args, out)
        model_management.unload_model(model)
        model_management.unload_model(out[1].patcher)
        sdxlrun.get_device_memory()
        refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
        model_management.unload_model(refinermodel)
        model_management.unload_model(refined_out[1].patcher)
        sdxlrun.get_device_memory()
        key+=1
